In [1]:
import requests
url = 'https://worldcup.sfg.io/matches'
r = requests.get(url)
all_games = r.json()

In [2]:
from datetime import datetime as dt
from sqlalchemy import Column, Integer, String, ForeignKey, DateTime, create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, backref

Base = declarative_base()

class Game(Base):
    __tablename__ = 'games'
    id = Column(Integer, primary_key=True)
    venue = Column(String)
    winner = Column(String)
    statistics = relationship('Statistics', back_populates="game")
    teams = relationship(
        'Team',
        secondary='statistics',
        back_populates='games'
    )

class Team(Base):
    __tablename__ = 'teams'
    id = Column(Integer, primary_key=True)
    country = Column(String)
    statistics = relationship('Statistics', back_populates="team")
    games = relationship(
        'Game',
        secondary='statistics',
        back_populates='teams'
    )

class Statistics(Base):
    __tablename__ = 'statistics'
    id = Column(Integer, primary_key=True)
    game_id = Column(Integer, ForeignKey('games.id'))
    team_id = Column(Integer, ForeignKey('teams.id'))
    goals = Column(Integer)
    game = relationship('Game', back_populates='statistics')
    team = relationship('Team', back_populates='statistics')
    ball_possession=Column(Integer)
    distance_covered=Column(Integer)
    on_target=Column(Integer)
    pass_accuracy=Column(Integer)

engine = create_engine('sqlite:///worldcup.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

In [3]:
all_games_tuples= [(game['venue'], game['home_team_country'], game['away_team_country'], game['winner']) for game in all_games]

In [4]:
h_teams = set([t[1] for t in all_games_tuples])
a_teams = set([t[2] for t in all_games_tuples])
teams = h_teams.union(a_teams)

In [5]:
team_objects = [Team(country = t) for t in teams]
len(team_objects)

32

In [6]:
session.add_all(team_objects)
session.commit()

In [7]:
all_game_objects = []
all_stats = []
for game in all_games:
    h_team = session.query(Team).filter(Team.country == game['home_team_statistics']['country']).one()
    a_team = session.query(Team).filter(Team.country == game['away_team_statistics']['country']).one()
    game_object = Game(venue= game['venue'],
                       teams = [h_team, a_team],
                       winner = game['winner'])
    all_game_objects.append(game_object)
    
    
    h_goals = game['home_team']['goals']
    
    h_ball_possession = game['home_team_statistics']['ball_possession']
    a_goals = game['away_team']['goals']
    a_ball_possession = game['away_team_statistics']['ball_possession']
    h_distance_covered = game['home_team_statistics']['distance_covered']
    a_distance_covered = game['away_team_statistics']['distance_covered']
    h_pass_accuracy = game['home_team_statistics']['pass_accuracy']
    a_pass_accuracy = game['away_team_statistics']['pass_accuracy']
    h_on_target = game['home_team_statistics']['on_target']
    a_on_target = game['away_team_statistics']['on_target']

# #     game = session.query(Game).filter(Game.teams[0].country == ['home_team_statistics']['country']).filter(Game.teams[1].country == ['away_team_statistics']['country']).one()
# # # # tried: game = session.query(Game).filter(Game.teams[0].country == stat[1]['country']).filter(Game.team[1].country == stat[3]['country']).one()
# # # #    
# # # this was a test. still need to get the correct game ; game = session.query(Game).filter(Game.venue == 'Moscow').all()[0]
   
    h_stats = Statistics(game = game_object,
                         team = h_team, 
                        ball_possession =h_ball_possession,
                        goals = h_goals,
                        distance_covered = h_distance_covered,
                        pass_accuracy = h_pass_accuracy,
                        on_target = h_on_target)
    a_stats = Statistics(game = game_object,
                        team = a_team,
                        ball_possession =a_ball_possession,
                        goals = a_goals,
                        distance_covered = a_distance_covered,
                        pass_accuracy = a_pass_accuracy,
                        on_target = a_on_target)
    all_stats.append(h_stats)
    all_stats.append(a_stats) 

In [37]:
all_game_objects[0].teams[1].country 

'Saudi Arabia'

In [7]:
# session.query(Team).filter(Team.country == 'Russia').one()

In [40]:
all_games[0]['home_team_statistics']

{'country': 'Russia',
 'attempts_on_goal': 13,
 'on_target': 7,
 'off_target': 3,
 'blocked': 3,
 'woodwork': 0,
 'corners': 6,
 'offsides': 3,
 'ball_possession': 40,
 'pass_accuracy': 78,
 'num_passes': 306,
 'passes_completed': 240,
 'distance_covered': 118,
 'balls_recovered': 53,
 'tackles': 9,
 'clearances': 19,
 'yellow_cards': 1,
 'red_cards': 0,
 'fouls_committed': 22,
 'tactics': '4-2-3-1',
 'starting_eleven': [{'name': 'Igor AKINFEEV',
   'captain': True,
   'shirt_number': 1,
   'position': 'Goalie'},
  {'name': 'MARIO FERNANDES',
   'captain': False,
   'shirt_number': 2,
   'position': 'Defender'},
  {'name': 'Ilya KUTEPOV',
   'captain': False,
   'shirt_number': 3,
   'position': 'Defender'},
  {'name': 'Sergey IGNASHEVICH',
   'captain': False,
   'shirt_number': 4,
   'position': 'Defender'},
  {'name': 'Iury GAZINSKY',
   'captain': False,
   'shirt_number': 8,
   'position': 'Midfield'},
  {'name': 'Alan DZAGOEV',
   'captain': False,
   'shirt_number': 9,
   'posit

In [38]:
all_stats = []
for g in all_games:
    h_goals = game['home_team']['goals']
    h_ball_possession = game['home_team_statistics']['ball_possession']
    a_goals = game['away_team']['goals']
    a_ball_possession = game['away_team_statistics']['ball_possession']
    h_distance_covered = game['home_team_statistics']['distance_covered']
    a_distance_covered = game['away_team_statistics']['distance_covered']
    h_pass_accuracy = game['home_team_statistics']['pass_accuracy']
    a_pass_accuracy = game['away_team_statistics']['pass_accuracy']
    h_on_target = game['home_team_statistics']['on_target']
    a_on_target = game['away_team_statistics']['on_target']
    game = session.query(Game).filter(Game.teams[0].country == g['home_team_statistics']['country'],
                                      Game.teams[1].country == g['away_team_statistics']['country']).one()
# # # # tried: game = session.query(Game).filter(Game.teams[0].country == 
# stat[1]['country']).filter(Game.team[1].country == stat[3]['country']).one()
# # # #    
# # # this was a test. still need to get the correct game ; game = session.query(Game).filter(Game.venue == 'Moscow').all()[0]
   
    h_stats = Statistics(game = game_object,
                         team = h_team, 
                        ball_possession =h_ball_possession,
                        goals = h_goals,
                        distance_covered = h_distance_covered,
                        pass_accuracy = h_pass_accuracy,
                        on_target = h_on_target)
    a_stats = Statistics(game = game_object,
                        team = a_team,
                        ball_possession =a_ball_possession,
                        goals = a_goals,
                        distance_covered = a_distance_covered,
                        pass_accuracy = a_pass_accuracy,
                        on_target = a_on_target)
    all_stats.append(h_stats)
    all_stats.append(a_stats) 

NotImplementedError: <built-in function getitem>

In [47]:
team_test = Team(country='Russia') 
session.query(Game).filter(Game.teams == team_test)

InvalidRequestError: Can't compare a collection to an object or collection; use contains() to test for membership.

In [48]:
session.query(Game).filter(Game.teams.country.all() == ['home_team_statistics']['country'], ['away_team_statistics']['country'])

AttributeError: Neither 'InstrumentedAttribute' object nor 'Comparator' object associated with Game.teams has an attribute 'country'

In [35]:
all_hs = [game['home_team_statistics']['country'] for game in all_games]
all_as = [game['away_team_statistics']['country'] for game in all_games]
all_as

['Saudi Arabia',
 'Uruguay',
 'Iran',
 'Spain',
 'Australia',
 'Iceland',
 'Denmark',
 'Nigeria',
 'Serbia',
 'Mexico',
 'Switzerland',
 'Korea Republic',
 'Panama',
 'England',
 'Japan',
 'Senegal',
 'Egypt',
 'Morocco',
 'Saudi Arabia',
 'Spain',
 'Australia',
 'Peru',
 'Croatia',
 'Costa Rica',
 'Iceland',
 'Switzerland',
 'Tunisia',
 'Mexico',
 'Sweden',
 'Panama',
 'Senegal',
 'Colombia',
 'Russia',
 'Egypt',
 'Morocco',
 'Portugal',
 'France',
 'Peru',
 'Croatia',
 'Argentina',
 'Germany',
 'Sweden',
 'Costa Rica',
 'Brazil',
 'Poland',
 'Colombia',
 'Belgium',
 'Tunisia',
 'Argentina',
 'Portugal',
 'Russia',
 'Denmark',
 'Mexico',
 'Japan',
 'Switzerland',
 'England',
 'France',
 'Belgium',
 'England',
 'Croatia',
 'Belgium',
 'England',
 'England',
 'Croatia']

In [14]:
game= all_game_objects[0]
#session.query(Game).filter(Game.teams[0].country == session.query(Team).filter(Team.country == game['home_team_statistics']['country']).one()['country'])
#session.query(Game).filter(Game.teams[0].country == 'Russia')
game.teams[0].country


'Russia'

In [10]:
# len(all_stats)

128

In [20]:
len(all_game_objects)

64

In [10]:
# assert(len(all_game_objects)==64)

In [8]:
session.add_all(all_game_objects)
session.add_all(all_stats)
session.commit()

In [12]:
session.query(Game).filter(Game.venue == 'St. Petersburg').all()[0].teams[0].country

'Morocco'

In [13]:
all_games[0]['home_team']['goals']

5

In [14]:
all_stats[29].game.venue

'Saransk'

In [17]:
len(session.query(Game).join(Statistics).filter_by(Team.country == 'Colombia').first()

SyntaxError: unexpected EOF while parsing (<ipython-input-17-4926ae262127>, line 1)

In [ ]:


#1. Just statistics objects
#2. Go through all of the stats objects
    all_stats = session.query(Stat).all()
    for stat in all_stats:
        name = stat.country_name
        team = session.query(Team).filter_by(name=name)
#         stat.team_id = team.id
        stat.team = team
        session.commit()
        
        

for stat in all_statistics:
    name = find_country_name(stat)
    team = session.query(Team).filter_by(name=name)
    stat.team_id = team.id
    